## Usage
- Edit the settings in the cell below.
- Cell -> Run All.
- A Napari window will open, where you can scroll through your data.

In [ ]:
import matplotlib.pyplot as plt
import napari
import numpy as np
import pandas as pd

from aicsimageio.aics_image import AICSImage
from pathlib import Path

# change this to point to your plate
plate = 'lyu/140324-A53T P62 staining/140324-A53T P62 staining/2024-03-27/20378/TimePoint_1'

# change lmu_active1 root folder depending on Linux vs. Windows
lmu_active1 = Path('/mnt/lmu_active1')
#lmu_active1 = Path('L:\lmu_active1')

# define colors you want to use (as many as you have channels)
#colormap = ["yellow", "magenta", "cyan"]
colormap = ["blue", "green", "red"]

## Code
You don't need to make changes in the code cells below. 

In [ ]:
# original image folder
orig = lmu_active1 / Path('instruments/Nano') / Path(plate)

# csv file with image sets (all channels on same row)
airflow = lmu_active1 / Path('airflow/nano')
plate_fixed = plate.replace(' ', '_')
csv = airflow / Path(plate_fixed) / Path('quality/Combined_MyExpt_Image.csv')

df = pd.read_csv(csv)
df.Metadata_Plate = df.Metadata_Plate.astype(str)
df.Metadata_Well = df.Metadata_Well.astype(str)
df.Metadata_Site = df.Metadata_Site.astype(int)
df.sort_values(by=['Metadata_Plate', 'Metadata_Well', 'Metadata_Site'], inplace=True, ignore_index=True)

In [ ]:
filename_cols = [c for c in df.columns if c.startswith('FileName')]
filename_cols

In [ ]:
wavelengths = [c.replace('FileName_', '') for c in filename_cols]
wavelengths

In [ ]:
metadata_cols = [c for c in df.columns if c.startswith('Metadata')]
metadata_cols

In [ ]:
filenames = df[filename_cols]
print(filenames.shape)
filenames.head()


In [ ]:
import dask.array as da
import numpy as np

def concat_da(selection, filename_cols, directory):
    da_out = None
    sites = []
    for index, row in selection.iterrows():
        channels = []
        for f in filename_cols:
            path = directory / row[f]
            #print(str(path))
            img = AICSImage(path)
            #channels.append(img.data)
            channels.append(img.get_image_dask_data())
            #print(img.get_image_dask_data().shape)
        site = da.concatenate(channels, axis=2)
        sites.append(site)

    da_out = da.concatenate(sites, axis=0)
    return da_out



In [ ]:
A_out = concat_da(filenames, filename_cols, orig)
print(A_out.shape)

In [ ]:
from qtpy.QtWidgets import QLabel, QWidget, QVBoxLayout

viewer = napari.view_image(
        A_out,
        channel_axis=2,
        name=wavelengths,
        colormap=colormap,
        contrast_limits=[[200, 4095], [500, 4095], [200, 4095]],
        )

# Create a custom widget to display the stack index
class StackIndexWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.label = QLabel("Current stack index: 0")
        #self.label_file = QLabel("Filename w1")
        self.label_plate = QLabel("Metadata_Plate: N/A")
        self.label_well = QLabel("Metadata_Well: N/A")
        self.label_site = QLabel("Metadata_Site: N/A")
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        #layout.addWidget(self.label_file)
        layout.addWidget(self.label_plate)
        layout.addWidget(self.label_well)
        layout.addWidget(self.label_site)
        self.setLayout(layout)

    def update_index(self, index):
        self.label.setText(f"Current stack index: {index}")
        #self.label_file.setText(f"Filename w1: {df.loc[index, 'FileName_w1']}")
        self.label_plate.setText(f"Metadata_Plate: {df.loc[index, 'Metadata_Plate']}")
        self.label_well.setText(f"Metadata_Well: {df.loc[index, 'Metadata_Well']}")
        self.label_site.setText(f"Metadata_Site: {df.loc[index, 'Metadata_Site']}")

# Instantiate the custom widget
stack_index_widget = StackIndexWidget()

# Add the widget to Napari as a dock widget
viewer.window.add_dock_widget(stack_index_widget, name="Stack Index")

# Callback function to update the widget with the current stack index
def on_index_change(event):
    current_index = viewer.dims.current_step[0]
    stack_index_widget.update_index(current_index)

# Connect the callback to the dims event
viewer.dims.events.current_step.connect(on_index_change)
